In [1]:
import pandas as pd

train = pd.read_csv('/kaggle/input/titanic-dataset/train.csv')
test = pd.read_csv('/kaggle/input/titanic-dataset/test.csv')

train.head()
train.info()
train.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [2]:
train.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
columns_to_drop = ['Cabin', 'Ticket', 'Name']
train = train.drop(columns=[col for col in columns_to_drop if col in train.columns])
test = test.drop(columns=[col for col in columns_to_drop if col in test.columns])


In [4]:
# Fill Age with median
train['Age'] = train['Age'].fillna(train['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())

# Fill Embarked with mode
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

# Fill Fare in test set
test['Fare'] = test['Fare'].fillna(test['Fare'].median())


In [5]:
# Convert Sex and Embarked to numeric using one-hot encoding
train = pd.get_dummies(train, columns=['Sex', 'Embarked'], drop_first=True)
test = pd.get_dummies(test, columns=['Sex', 'Embarked'], drop_first=True)


In [6]:
# Fill Age
if 'Age' in train.columns:
    train['Age'] = train['Age'].fillna(train['Age'].median())
if 'Age' in test.columns:
    test['Age'] = test['Age'].fillna(test['Age'].median())

# Fill Embarked (only if exists)
if 'Embarked' in train.columns:
    train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

# Fill Fare
if 'Fare' in test.columns:
    test['Fare'] = test['Fare'].fillna(test['Fare'].median())


In [7]:
# One-hot encode safely
categorical_cols = ['Sex', 'Embarked']
for col in categorical_cols:
    if col in train.columns:
        train = pd.get_dummies(train, columns=[col], drop_first=True)
    if col in test.columns:
        test = pd.get_dummies(test, columns=[col], drop_first=True)


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Show column names to pick correct features
print(train.columns)


Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Sex_male', 'Embarked_Q', 'Embarked_S'],
      dtype='object')


In [9]:
# Use only available features from the list above
features = [col for col in ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_male', 'Embarked_Q', 'Embarked_S'] if col in train.columns]

X = train[features]
y = train['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))


Validation Accuracy: 0.8212290502793296


In [10]:
# Ensure test set has the same features
X_test = test[features]  # use same 'features' list from training

# Predict
test_preds = model.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_preds
})

submission.to_csv('submission.csv', index=False)
print("✅ submission.csv created!")


✅ submission.csv created!


In [11]:
submission.to_csv('submission.csv', index=False)
